# Approaching Time Series Forecasting Problems 🗓

The goal of this lesson is to help students think through how to approach a time series problem.

## Learning Objectives
By the end of the lesson students should be familiar with:

- Holt-Winters Triple Exponential Smoothing
- SARIMAX
- auto-ARIMA

## Before modeling
- Get the time period into *datetime* dtype format and get it in the index
- Become comfortable with resampling and other pandas time series methods

## Modeling considerations
- Make sure you aren't leaking information from the future into your training data
- Make a null model (just the most recent prediction continued) first, so you have a baseline
- There are a lot of model possibilities of various complexity levels (e.g. weighted average, simple exponential smoothing, ARIMA)
- Consider the tradeoffs between complexity and performance when choosing which model to use

## Models that can handle more complexity well
- Holt-Winters (exponential smoothing)
- Prophet 
- SARIMAX 
- VAR
- Deep Neural Networks

## Holt-Winters (exponential smoothing)
[Holt Winters](https://www.statsmodels.org/dev/generated/statsmodels.tsa.holtwinters.ExponentialSmoothing.html) often performs about as well as SARIMA. It requires much less tuning. It's part of the exponential smoothing family of algorithms. 

- It's also called *Triple Exponential Smoothing*

- You don't have to worry about stationarity. 

- It's available in statsmodels [here](https://www.statsmodels.org/dev/generated/statsmodels.tsa.statespace.exponential_smoothing.ExponentialSmoothing.html#statsmodels.tsa.statespace.exponential_smoothing.ExponentialSmoothing). 

Simple exponential smoothing uses weighted averages where the weights decrease exponentially for older data points.

Holt-Winters is more powerful because it accounts for seasonality and trend. 

The model is made up of trend, level, and seasonal components.

If the outcome variable is sales:

* The trend equation captures the overall direction of sales. 
* The level equation is a weighted average between the seasonally adjusted observation and the non-seasonal forecast for time `t`. 
* The seasonal equation is a weighted average between the current seasonal index and the seasonal index of the same season `s` time periods ago. 

You have to choose the number of seasonal periods, and whether there are additive,  multiplicative, or damping.

You can also pass exogenous variables.

Here's what it looks like in code.

In [ ]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels

In [ ]:
statsmodels.__version__

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

holt_winters = ExponentialSmoothing(
    (train['Weekly_Sales'].to_numpy()),  # note have to feed it numpy ndarray
    trend='mul', 
    seasonal='mul',
    seasonal_periods=52,

).fit()

df_predictions['Holt_Winters'] = holt_winters.forecast(len(test))


In [ ]:
plt.figure(figsize=(16,8))
plt.plot( train['Weekly_Sales'], label='Train')
plt.plot(test['Weekly_Sales'], label='Test')
plt.plot(df_predictions['Holt_Winters'], label='Holt_Winter')
plt.legend(loc='best')
plt.title("Holt Winters!")
plt.show()

What do you think of that curve?

Let's look at the RMSE to check accuracy.

In [ ]:
rms = mean_squared_error(
    test['Weekly_Sales'], 
    df_predictions['Holt_Winters'], 
    squared=False
)
print(f'{rms:,.0f}')

## Prophet
Facebook created and opensourced the [Prophet](https://facebook.github.io/prophet/) library. 

[Here's](https://medium.com/future-vision/the-math-of-prophet-46864fa9c55a) a good article on what Prophet is doing.
"Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects." 

An additive model is a nonparametric regression method.

Prophet requires much less tuning than SARIMA. Facebook finds it very helpful for advertising type data and it's convenient for adding things like holidays. It's relatively new and it's popular. 

## SARIMAX
You know ARIMA. 

SARIMA adds seasonality (and you get to choose 4 other parameters) 😀. 

There are 7 parameters to choose `(p, d, q)` `(P, D, Q)` and `m`. `p` is for the autoregressive component, `i` stands for *integrative* and is all about differencing, `q` is for the moving average component. The next three capitalized arguments are the same but for the Seasonality component. And `m` is for the number of observations per seasonal cylce. With weekly sales data with a holiday sesonality, that's would be 52 weeks, generally.

SARIMAX extends SARIMA. The X in [SARIMAX](https://www.statsmodels.org/dev/examples/notebooks/generated/statespace_sarimax_stata.html) stands for eXogenous variables. It allows you to add predictors that aren't just the prevoious outcome variable. For example, you can dummy encode a column to be a holiday/no holiday, (1/0).

## VAR

VAR is useful when you have multiple time series you want to include in your forecast.

VAR was touched on briefly in global and Mahdi has a whole repo [here](https://git.generalassemb.ly/DSI-US-11/local_var).

## Neural Networks
Deep neural networks can be used with time series data. There are a number of of architectures that can make sense. [Here's](https://towardsdatascience.com/deep-learning-for-time-series-and-why-deep-learning-a6120b147d60) a post on the topic if you are interested. If the relationship between the input variables and the outcome is not complex, this is probably overkill.

## Putting it all together

If you find a model that works really well, great!

If you are trying to get every last bit of performance, you can also ensemble your models. I have not done this.

## Next steps
I recommend checking out the great free ebook [Forecasting: Principles and Practice
Rob J Hyndman and George Athanasopoulos](https://otexts.com/fpp3/) to learn more about time series. 

Jason Brownlee aslo has a lot of great information and Python code on time series forecasting at [Machine Learning Mastery](https://machinelearningmastery.com/exponential-smoothing-for-time-series-forecasting-in-python/).

# Example: Store sales data

#### Read in the dataset, set the index and take a look at it.

In [ ]:
df_walmart = pd.read_csv('data/train.csv', ...)

Alternatively, we could have set the index after creating the DataFrame.

In [ ]:
# df_walmart = df_walmart.set_index('Date') 
df_walmart.head()

Let's make sure that index is a `datetime`.

#### Filter the DataFrame to Store 1 sales and aggregate over departments to compute the total sales per store.

In [ ]:
store1

In [ ]:
store1_sales.head()

Let's make that a DataFrame.

In [ ]:
df_store1_sales = pd.DataFrame(store1_sales)

In [ ]:
df_store1_sales.head()

In [ ]:
df_store1_sales.info()

Let's plot it.

In [ ]:
df_store1_sales.plot();

### Split the dataset into train and test sets.

We will subset the data to look at the accuracy of each time series forecasting method on the data set. We'll use the first two years (2010–2011) as the "training" data and the last year (2012) as the "testing" data for the purposes of our demonstration. 

Let's make two DataFrames.

In [ ]:
train = df_store1_sales['2010': '2011']
test = df_store1_sales['2012']

In [ ]:
train.head()

In [ ]:
test.head()

Let's plot the two DataFrames

In [ ]:
train['Weekly_Sales'].plot(figsize=(15,8))
test['Weekly_Sales'].plot(figsize=(15,8), title= 'Weekly Sales', fontsize=14)
plt.show()

## Method 1: Naive Forecast

Take the last time period's sales and estimate the same value for all future time periods. This method is called a **naive forecast**.

$${\Large \hat y_{t+1} = y_t}$$

#### Let's see how well the naive method does when forecasting sales.

Make a predictions DataFrame and set the predicted values equal to the last value in the `Weekly_Sales` df.

In [ ]:
train.tail(2)

In [ ]:
df_predictions = test.copy()  
df_predictions['naive'] = train['Weekly_Sales'].iloc[-1]
df_predictions

Plot the historic data, the actual "future" data, and the predicted "future" data.

In [ ]:
plt.figure(figsize=(12,8))

plt.plot(train.index, train['Weekly_Sales'], label='Train')
plt.plot(test.index, test['Weekly_Sales'], label='Test')
plt.plot(df_predictions.index, df_predictions['naive'], label='Naive Forecast')

plt.legend(loc='best')
plt.title("Naive Forecast")
plt.show()

Let's check the model's performance on the test data set using the Root Mean Squared Error (RMSE) scoring metric. 

This `mean_squared_error metric` function can be imported from Scikit-learn (sklearn). 

In [ ]:
from sklearn.metrics import mean_squared_error

### Get the RMSE for our baseline

That's our baseline. 

We can infer from the RMSE value and the graph above that the naive method isn’t amazing for data sets with high variability. It's best suited for stable data sets. Let's try to improve the score with more sophisticated models. 

Note that we're looking at a prediction through the end of the time period as of a single date, we aren't assuming we're updating the model every week with new information. 

## SARIMA with Auto-arima package: pmdarima

There are a bunch of ways to find parameter values for SARIMA. You can use autocorrelation plots and partial correlation plots to try to figure out parameters, but this is inexact and tedious. You can make your own GridSearching function. Or you, can use a nice one out of the box.

Let's do some statistical testing and search through parameters automatically the `pdarima.auto_arima` library. 

The Auto-arima algorithm can grid search for `p` and `q`. It doesn't help with `d`. So we'll figure out the differencing hyperparameter ourselves.

If you want to use this library in the future, install with pip because the conda instructions didn't work as of 2/1/20.

From the command line run: 
`pip install pmdarima`

I imported the pmdarima package under the alias `pm`. It has some helpful functions and will pick parameters and create a model using statsmodels under the hood.

In [ ]:
import pmdarima as pmd

Let's make an autocorrelation plot just for fun.

Test for what the differencing coefficient (d) should be. Instructions from https://alkaline-ml.com/pmdarima/tips_and_tricks.html

In [ ]:
ndiffs = pmd.arima.utils.ndiffs   

# Estimate the number of differences using an ADF (Augmented Dickey Fuller) test:


No differencing needed. Nice. 😀

`m=52` for the number of observations per seasonal cycle

Let's instantiate a model with a whole lot of arguments.

Running this next cell might take some time. The `summary)` method is gridsearching, fitting lots of different models to the training data to see which hyperparameters work best.

The Model displays the final parameters. Now we can the model object to predict the "future" values.

In [ ]:
plt.figure(figsize=(16,8))
plt.plot( train['Weekly_Sales'], label='Train')
plt.plot(test['Weekly_Sales'], label='Test')
plt.plot(df_predictions['SARIMA_auto'], label='SARIMA_auto')
plt.legend(loc='best')
plt.show()

In [ ]:
rms = mean_squared_error(
    test['Weekly_Sales'], 
    df_predictions['SARIMA_auto'],
    squared=False
)
print(f'{rms:,.0f}')

## That's all the time we have today! 😂